In [1]:
import sys
sys.path.insert(1, '/home/amrzr/Work/Codes/Offline_IMOEA_Framework/')

from desdeo_problem.Problem import DataProblem
from main_project_files.surrogate_fullGP import FullGPRegressor as fgp
from main_project_files.surrogate_SVR import SVRsurrogate
from main_project_files.surrogate_RandomForest import RandomForest
from main_project_files.surrogate_PolynomialRegression import PolynomialRegressionsurrogate as polyreg
from sklearn.model_selection import KFold, LeaveOneOut
import time
import numpy as np
import pandas as pd

In [2]:
nobjs = 3 
nvars = 22

main_directory = 'Pump_Test_Tomas_6_177'
data_folder = '/home/amrzr/Work/Codes/data'
#data_file = data_folder+'/pump_data/01_DOE_data.csv'
#data_file = data_folder+'/pump_data/02_DOE_140_data.csv'
data_file = data_folder+'/pump_data/initial_all_amz.csv'
#data_file = data_folder+'/pump_data/03_DOE_140_all_data.csv'
path = data_folder + '/test_runs/' + main_directory

df = pd.read_csv(data_file)
df[['f1','f2','f3']] = df[['f1','f2','f3']]*-1

#x_low = np.min(zz,axis=0)
#x_high = np.max(zz,axis=0)
#x_low = [20, 0.2, 0.22, 0.25, -5, 85, 355, 450, 15, 15, -10, 16, 0.25, 0.2, 0.25, -5, 85, 450, 15, 15, 27, -15]
#x_high = [30,0.72, 0.76 , 0.8 , 0 , 90, 380 , 600, 45 , 50,10 , 26, 0.76, 0.7 ,0.76, 0 ,90 ,600 ,60 ,50 ,35 ,5]
x_low = np.ones(22)*0
x_high = np.ones(22)
x_names = list(df.columns)[0:22]
y_names = list(df.columns)[22:25]
row_names = ['lower_bound','upper_bound']

bounds = pd.DataFrame(np.vstack((x_low,x_high)), columns=x_names, index=row_names)


In [17]:
def K_fold_test_surrogate(df, surrogate):
    splits=10
    kf=KFold(n_splits=splits,shuffle=True,random_state=5)
    M_RMSE=np.zeros([splits,3])
    M_R2=np.zeros([splits,3])
    i=0

    for trainIndex, testIndex in kf.split(df):
        # print(trainIndex, testIndex)
        
        dfTrain = df.loc[trainIndex, :]
        dfTest = df.loc[testIndex, :]
        xTest = dfTest.values[:,0:22]
        N=len(dfTest)
        Q_train = dfTrain[['f1','f2','f3']]
        Q_test = dfTest[['f1','f2','f3']]
        Qminus_mean=Q_test-np.mean(Q_train,0)
        ss_tot=np.diag(np.dot(Qminus_mean.T,Qminus_mean))
        #print("sstot=",ss_tot)
        problem = DataProblem(data=dfTrain, variable_names=x_names, objective_names=y_names,bounds=bounds)
        problem.train(surrogate)

        Y_pred = problem.evaluate(xTest,use_surrogate=True)[0]
            
        surr_error=np.abs(Y_pred-dfTest[['f1','f2','f3']])
        #print('surrogate error is',surr_error)
        ss_res=np.diag(np.dot(surr_error.T,surr_error))
        #print("ss_res=",ss_res)
        RMSE=np.sqrt(ss_res/N)
        print('RMSE_surr:',RMSE)
        print('RMSE_mean:',np.sqrt(ss_tot/N))
        M_RMSE[i,:]=RMSE
        R2score=1-ss_res/ss_tot
        M_R2[i,:]=R2score
        i=i+1
    
    avr_RMSE=np.median(M_RMSE,0)
    print('median RMSE is',avr_RMSE)

    avr_R2score=np.median(M_R2,0)
    print('median R2score is',avr_R2score)


In [19]:
K_fold_test_surrogate(df, RandomForest)

RMSE_surr: [1.67450895 2.90011692 5.65369653]
RMSE_mean: [2.04949988 2.39989052 3.20129991]
RMSE_surr: [1.60954954 1.85881573 2.41103309]
RMSE_mean: [2.23616046 2.07340095 2.80755374]
RMSE_surr: [2.19165649 4.60561487 5.38103344]
RMSE_mean: [2.76620187 3.53747963 4.69945213]
RMSE_surr: [1.48190675 2.12854794 3.43027099]
RMSE_mean: [1.80336703 2.054977   3.84262812]
RMSE_surr: [1.30505737 3.0625023  4.55636747]
RMSE_mean: [1.86261814 3.3013212  5.7595041 ]
RMSE_surr: [ 3.36879275  7.24567039 13.03340953]
RMSE_mean: [ 3.29841631  7.30344686 14.00808468]
RMSE_surr: [1.40114319 1.8694312  2.52070395]
RMSE_mean: [1.88984332 2.27983494 3.67968964]
RMSE_surr: [1.67844002 2.31474959 3.3138699 ]
RMSE_mean: [2.00754168 2.76316814 4.85404412]
RMSE_surr: [1.29913053 2.40032597 3.79234931]
RMSE_mean: [1.15791211 2.80615794 5.63387542]
RMSE_surr: [1.16076888 1.51894305 2.4782934 ]
RMSE_mean: [1.49550852 1.78104612 2.48668486]
median RMSE is [1.54572814 2.35753778 3.61131015]
median R2score is [0.352

In [18]:
K_fold_test_surrogate(df, SVRsurrogate)

RMSE_surr: [1.53248451 2.23647845 2.85873719]
RMSE_mean: [2.04949988 2.39989052 3.20129991]
RMSE_surr: [1.71261399 1.626558   2.03932818]
RMSE_mean: [2.23616046 2.07340095 2.80755374]
RMSE_surr: [2.02880212 3.13774407 4.44281426]
RMSE_mean: [2.76620187 3.53747963 4.69945213]
RMSE_surr: [1.16519655 2.10392096 3.56344271]
RMSE_mean: [1.80336703 2.054977   3.84262812]
RMSE_surr: [1.24609637 3.11290232 5.17634752]
RMSE_mean: [1.86261814 3.3013212  5.7595041 ]
RMSE_surr: [ 3.1703217   7.22208347 13.85620855]
RMSE_mean: [ 3.29841631  7.30344686 14.00808468]
RMSE_surr: [1.3653195  1.70447222 2.26633685]
RMSE_mean: [1.88984332 2.27983494 3.67968964]
RMSE_surr: [1.53497805 2.48583719 4.68129212]
RMSE_mean: [2.00754168 2.76316814 4.85404412]
RMSE_surr: [0.95445637 2.58866289 5.09446389]
RMSE_mean: [1.15791211 2.80615794 5.63387542]
RMSE_surr: [1.00067596 1.26701347 1.50740774]
RMSE_mean: [1.49550852 1.78104612 2.48668486]
median RMSE is [1.448902   2.36115782 4.00312848]
median R2score is [0.451

In [20]:
K_fold_test_surrogate(df, polyreg)

RMSE_surr: [1.19342519 2.17507515 3.27377559]
RMSE_mean: [2.04949988 2.39989052 3.20129991]
RMSE_surr: [1.60311774 2.4958465  4.7583968 ]
RMSE_mean: [2.23616046 2.07340095 2.80755374]
RMSE_surr: [1.5796401  2.65740014 4.21763073]
RMSE_mean: [2.76620187 3.53747963 4.69945213]
RMSE_surr: [1.05888154 1.89536185 4.01147187]
RMSE_mean: [1.80336703 2.054977   3.84262812]
RMSE_surr: [1.61866539 2.63333512 3.82173906]
RMSE_mean: [1.86261814 3.3013212  5.7595041 ]
RMSE_surr: [ 3.01282545  6.02217496 12.11926907]
RMSE_mean: [ 3.29841631  7.30344686 14.00808468]
RMSE_surr: [ 2.92440504  4.99394996 10.81625554]
RMSE_mean: [1.88984332 2.27983494 3.67968964]
RMSE_surr: [1.75315767 2.79004749 4.73152937]
RMSE_mean: [2.00754168 2.76316814 4.85404412]
RMSE_surr: [1.5615742  3.02206529 4.09941836]
RMSE_mean: [1.15791211 2.80615794 5.63387542]
RMSE_surr: [1.02927133 2.19411956 2.9433182 ]
RMSE_mean: [1.49550852 1.78104612 2.48668486]
median RMSE is [1.59137892 2.64536763 4.15852454]
median R2score is [0.

In [21]:
K_fold_test_surrogate(df, fgp)

 /home/amrzr/Work/Codes/.venvs/Offline_IMOEA_Framework/lib/python3.8/site-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:overflow encountered in square
 /home/amrzr/Work/Codes/.venvs/Offline_IMOEA_Framework/lib/python3.8/site-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add


RMSE_surr: [1.3115541  2.52833615 3.46295301]
RMSE_mean: [2.04949988 2.39989052 3.20129991]
RMSE_surr: [1.7283015  2.61538864 4.63778658]
RMSE_mean: [2.23616046 2.07340095 2.80755374]
RMSE_surr: [2.05098191 2.53665156 3.07963224]
RMSE_mean: [2.76620187 3.53747963 4.69945213]
RMSE_surr: [1.82808228 1.75247493 2.48540272]
RMSE_mean: [1.80336703 2.054977   3.84262812]
RMSE_surr: [1.36547446 2.2705557  3.47959545]
RMSE_mean: [1.86261814 3.3013212  5.7595041 ]
RMSE_surr: [ 3.29477422  6.4389054  12.07084635]
RMSE_mean: [ 3.29841631  7.30344686 14.00808468]
RMSE_surr: [1.59382243 1.64852201 4.62998678]
RMSE_mean: [1.88984332 2.27983494 3.67968964]
RMSE_surr: [1.80468682 2.89090958 3.35575035]
RMSE_mean: [2.00754168 2.76316814 4.85404412]
RMSE_surr: [1.60277596 2.64764284 2.4966542 ]
RMSE_mean: [1.15791211 2.80615794 5.63387542]
RMSE_surr: [1.23816944 1.69314832 3.04631721]
RMSE_mean: [1.49550852 1.78104612 2.48668486]
median RMSE is [1.66553873 2.53249385 3.40935168]
median R2score is [0.301

In [3]:
import GPy
import numpy as np
import pandas as pd

In [4]:

main_directory = 'Pump_Test_Tomas_6_177'
data_folder = '/home/amrzr/Work/Codes/data'
#data_file = data_folder+'/pump_data/sim_stat2.csv'
data_file = data_folder+'/pump_data/03_DOE_180_failed.csv'
df = pd.read_csv(data_file)   
X=df.values[:,0:22]
y=df.values[:,22]

labels = list(set(y.flatten()))
models = {}
for label in labels:
    ytmp=y.copy()
    ytmp[ytmp!=label]=0
    ytmp[ytmp==label]=1
    
    m=GPy.models.GPClassification(X, ytmp[:, None])
    
    m.optimize_restarts(messages=True, robust=True, 
                        num_restarts=5)
    #    else:
    #        m.optimize(messages=True)
    models[label]=m


Optimization restart 1/5, f = -0.36771524762052366


Optimization restart 2/5, f = 8.2614477359


Optimization restart 3/5, f = -2.2046470346533624


Optimization restart 4/5, f = -0.1990939841175532


Optimization restart 5/5, f = 13.047525346102873


Optimization restart 1/5, f = 85.93401686989367


Optimization restart 2/5, f = 124.76649250079018


Optimization restart 3/5, f = 82.74297028193396


Optimization restart 4/5, f = 87.99308297767817


Optimization restart 5/5, f = 89.99055457001698


In [10]:
models[1].predict(X)[0]<0.5

array([[ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [